In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Set dataset path (update folder name as needed)
data_dir = "/content/drive/MyDrive/data"


In [16]:
# Check dataset structure
!ls $data_dir

train  val


In [17]:
!pip install torch torchvision --quiet

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [20]:
# STEP 3: DATA TRANSFORMS & LOADERS
# ==============================
batch_size = 32

data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [21]:
# Load datasets
image_datasets = {
    x: datasets.ImageFolder(f"{data_dir}/{x}", data_transforms[x])
    for x in ['train', 'val']
}

In [22]:
# Create dataloaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
    for x in ['train', 'val']
}


In [23]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print("Classes:", class_names)
print("Train size:", dataset_sizes['train'], "Validation size:", dataset_sizes['val'])

Classes: ['medical', 'non-medical']
Train size: 3700 Validation size: 450


In [24]:
# STEP 4: MODEL - EfficientNet-B0
# ==============================
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Unfreeze last few layers for better feature learning
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-3:].parameters():
    param.requires_grad = True

# Replace classifier
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 133MB/s] 


In [25]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 20)
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model

In [26]:
# STEP 6: TRAIN MODEL
# ==============================
model = train_model(model, criterion, optimizer, num_epochs=3)

Epoch 1/3
--------------------
train Loss: 0.1464 Acc: 0.9643
val Loss: 0.0582 Acc: 0.9889
Epoch 2/3
--------------------
train Loss: 0.0493 Acc: 0.9892
val Loss: 0.0361 Acc: 0.9978
Epoch 3/3
--------------------
train Loss: 0.0424 Acc: 0.9886
val Loss: 0.0269 Acc: 0.9978


In [27]:
torch.save(model.state_dict(), "/content/drive/MyDrive/medical_vs_nonmedical_efficientnet_finetuned_1.pth")
print("Model saved.")

Model saved.


In [31]:
# STEP 7: PREDICTION FUNCTION
# ==============================
def predict_image(image_path):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    transform = data_transforms['val']
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, pred = torch.max(outputs, 1)
        return class_names[pred]

In [38]:
test_image_path = "/content/drive/MyDrive/football.jpg"
print(predict_image(test_image_path))

non-medical


In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
!pip install torch torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.5 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architecture
model = models.efficientnet_b0(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

# Load previously trained weights
model.load_state_dict(torch.load("/content/drive/MyDrive/medical_vs_nonmedical_efficientnet_finetuned.pth", map_location=device))
model = model.to(device)


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transforms for augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

data_dir = "/content/drive/MyDrive/new_data"

# Load dataset (no validation split)
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

class_names = dataset.classes
print("Classes:", class_names)
print("Dataset size:", len(dataset))


Classes: ['medical', 'non-medical']
Dataset size: 200


In [5]:
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-3:].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True


In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

def fine_tune(model, dataloader, epochs=3):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")
    return model

model = fine_tune(model, dataloader, epochs=3)



Epoch 1/3 - Loss: 0.6362, Acc: 0.7850
Epoch 2/3 - Loss: 0.1043, Acc: 0.9700
Epoch 3/3 - Loss: 0.0504, Acc: 0.9800


In [7]:
# Save updated model
torch.save(model.state_dict(), "/content/drive/MyDrive/medical_vs_nonmedical_efficientnet_finetuned_2.pth")
print("Fine-tuning complete.")

Fine-tuning complete.


In [13]:
from PIL import Image
from torchvision import transforms

# Same transforms as validation
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def predict_image(model, image_path):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img_t = test_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_t)
        _, pred = torch.max(outputs, 1)
    return class_names[pred]

print(predict_image(model, "/content/drive/MyDrive/knee.jpeg"))
# print(predict_image(model, "/content/drive/MyDrive/new_data/non-medical/car1.jpg"))


non-medical
